## 問題

https://twitter.com/e869120/status/1388262816101007363

## 解説

https://twitter.com/e869120/status/1388674962823663619

In [1]:
#r "nuget: FSharpPlus"

Installed Packages FSharpPlus, 1.3.2

In [2]:
open System
open System.Numerics
open System.Collections.Generic
open FSharpPlus

In [3]:
let solveUsingCompression W Ns =
  let Ns = Ns |> Seq.toList
  let positionDict = Ns |> Seq.collect (fun (l, r) -> [l; r]) |> Seq.distinct |> Seq.sort |> Seq.mapi (fun i x -> x, i) |> dict
  let heights = Array.zeroCreate<int> positionDict.Count
  Ns |> List.map (fun (l, r) ->
    let height = seq { l .. r } |> Seq.choose (positionDict.TryGetValue >> Option.ofPair >> Option.map (Array.get heights)) |> Seq.max |> (+) 1
    seq { l .. r } |> Seq.iter (positionDict.TryGetValue >> Option.ofPair >> Option.iter (fun i -> heights[i] <- height))
    height)

In [4]:
solveUsingCompression 100 [
  27, 100
  8, 39
  83, 97
  24, 75
]

index value 0 1 1 2 2 2 3 3

In [5]:
solveUsingCompression 3 [
  1, 2
  2, 2
  2, 3
  3, 3
  1, 2
]

index value 0 1 1 2 2 3 3 4 4 4

In [6]:
solveUsingCompression 10 [
  1, 3
  3, 5
  5, 7
  7, 9
  2, 4
  4, 6
  6, 8
  3, 5
  5, 7
  4, 6
]

index value 0 1 1 2 2 3 3 4 4 3 5 4 6 5 7 5 8 6 9 7

In [7]:
solveUsingCompression 500000 [
  1, 500000
  500000, 500000
  1, 500000
  1, 1
  1, 500000
  500000, 500000
  1, 500000
]

index value 0 1 1 2 2 3 3 4 4 5 5 6 6 7

In [8]:
/// 遅延評価セグメント木
/// count: データ数
/// et: データの単位元
/// ft: データに対する演算
/// em: 中間データの単位元
/// fm: 中間データに対する演算
/// ftm: データと中間データに対する演算
type LazySegmentTree<'T, 'M when 'T: equality and 'M: equality>(
  count: int,
  et: 'T,
  ft: ('T -> 'T -> 'T),
  em: 'M,
  fm: 'M -> 'M -> 'M,
  ftm: ('T -> 'M -> 'T)
) =
  let leafCount = Seq.initInfinite (pown 2) |> Seq.find (flip (>=) count)
  let nodeCount = leafCount * 2 + 1
  let nodes = Array.create nodeCount et
  let lazyNodes = Array.create nodeCount em
  let leftChildIndex i = 2 * i + 1
  let rightChildIndex i = 2 * i + 2
  let parentIndex i = (i - 1) / 2
  let leafIndex i = i + leafCount - 1
  let eval i =
    if lazyNodes[i] = em then () else
    if i < leafCount - 1 then
      lazyNodes[leftChildIndex i] <- fm lazyNodes[leftChildIndex i] lazyNodes[i]
      lazyNodes[rightChildIndex i] <- fm lazyNodes[rightChildIndex i] lazyNodes[i]
    nodes[i] <- ftm nodes[i] lazyNodes[i]
    lazyNodes[i] <- em
  let rec update a b x k l r =
    eval k
    if a <= l && r <= b then
      lazyNodes[k] <- fm lazyNodes[k] x
      eval k
    elif a < r && l < b then
      update a b x (leftChildIndex k) l ((l + r) / 2)
      update a b x (rightChildIndex k) ((l + r) / 2) r
      nodes[k] <- ft nodes[leftChildIndex k] nodes[rightChildIndex k]
  let rec query a b k l r =
    eval k
    if r <= a || b <= l then et
    elif a <= l && r <= b then nodes[k] else
    let vl = query a b (leftChildIndex k) l ((l + r) / 2)
    let vr = query a b (rightChildIndex k) ((l + r) / 2) r
    ft vl vr
  new(values: 'T list, et, ft, em, fm, ftm) as this =
    LazySegmentTree(length values, et, ft, em, fm, ftm)
    then this.Set values
  member private _.Set values =
    values |> List.iteri (fun i v -> nodes[leafIndex i] <- v)
    for i = leafCount - 2 downto 0 do nodes[i] <- ft nodes[leftChildIndex i] nodes[rightChildIndex i]
  member _.Count = count
  member _.Update a b x = update a b x 0 0 leafCount
  member _.Query a b = query a b 0 0 leafCount

In [10]:
let solveUsingSegmentTree W Ns =
  let Ns = Ns |> Seq.toList
  let positionDict = Ns |> Seq.collect (fun (l, r) -> [l; r]) |> Seq.distinct |> Seq.sort |> Seq.mapi (fun i x -> x, i) |> dict
  let segmentTree = LazySegmentTree(positionDict.Count, 0, max, 0, (fun _ m -> m), (fun _ m -> m))
  Ns |> List.map (fun (l, r) ->
    let li, ri = positionDict[l], positionDict[r] + 1
    let height = segmentTree.Query li ri + 1
    segmentTree.Update li ri height
    height)

In [11]:
solveUsingSegmentTree 100 [
  27, 100
  8, 39
  83, 97
  24, 75
]

index value 0 1 1 2 2 2 3 3

In [12]:
solveUsingSegmentTree 3 [
  1, 2
  2, 2
  2, 3
  3, 3
  1, 2
]

index value 0 1 1 2 2 3 3 4 4 4

In [13]:
solveUsingSegmentTree 10 [
  1, 3
  3, 5
  5, 7
  7, 9
  2, 4
  4, 6
  6, 8
  3, 5
  5, 7
  4, 6
]

index value 0 1 1 2 2 3 3 4 4 3 5 4 6 5 7 5 8 6 9 7

In [14]:
solveUsingSegmentTree 500000 [
  1, 500000
  500000, 500000
  1, 500000
  1, 1
  1, 500000
  500000, 500000
  1, 500000
]

index value 0 1 1 2 2 3 3 4 4 5 5 6 6 7